In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras import backend as K

In [0]:
DATADIR = '/content/drive/My Drive/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]


In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dirs = os.listdir('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')

In [74]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
X = []
Y = []
dirs = os.listdir('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
#dirs=dirs[:-1]
print(len(dirs))
'''

'''

21


'\n\n'

In [75]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and n<90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			X.append([r,g,b])
			Y.append([label])
			n = n+1
	print(n)
	label = label + 1

90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90


In [76]:
import numpy as np
x_bck=X
y_bck=Y


X =  np.array(X)
Y =  np.array(Y)
X = X.reshape(X.shape[0], img_rows, img_cols, 3)
X.shape


(1890, 256, 256, 3)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

In [0]:
import time 
import cv2 
from scipy.fftpack import dct 

a_256=[]
b_256=[]
#y = np.random.random((5000, 6, 6, 3))
t1 = time.time()
a_256 = np.zeros((len(X_train), 256, 256, 3))
b_256 = np.zeros((len(X_test), 256, 256, 3))
for i in range(len(X_train)):
    for j in range(3):
        a_256[i, : , : , j] = cv2.dct(X_train[i, : , : , j])
        np.transpose(a_256, [0, 2, 1, 3])

for i in range(len(X_train)):
    for j in range(3):
        a_256[i, :, :, j] = cv2.dct(X_train[i, :, :, j]) 

for i in range(0,len(X_test)):
    for j in range(3):
        b_256[i, : , : , j] = cv2.dct(X_test[i, : , : , j])
        np.transpose(b_256, [0, 2, 1, 3])

for i in range(len(X_test)):
    for j in range(3):
        b_256[i, :, :, j] = cv2.dct(X_test[i, :, :, j]) 
 


In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

##New model

In [0]:
model_big = []

In [0]:
model_big = Sequential()
model_big.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model_big.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Dropout(0.25))
model_big.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Dropout(0.25))
model_big.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Flatten())
model_big.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)))
model_big.add(Dropout(0.5))
model_big.add(Dense(21, activation='softmax'))
model_big.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [0]:
model_big.fit(a_256, y_train, batch_size=50, nb_epoch=150, verbose=1, validation_data=(b_256, y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 1512 samples, validate on 378 samples
Epoch 1/150
1512/1512 [==============================] - 28s 19ms/step - loss: 3.4531 - accuracy: 0.0893 - val_loss: 3.2007 - val_accuracy: 0.1429
Epoch 2/150
1512/1512 [==============================] - 16s 11ms/step - loss: 3.0417 - accuracy: 0.1541 - val_loss: 2.7548 - val_accuracy: 0.2063
Epoch 3/150
1512/1512 [==============================] - 16s 11ms/step - loss: 2.7086 - accuracy: 0.2480 - val_loss: 2.5577 - val_accuracy: 0.2778
Epoch 4/150
1512/1512 [==============================] - 16s 11ms/step - loss: 2.4402 - accuracy: 0.3241 - val_loss: 2.3542 - val_accuracy: 0.3757
Epoch 5/150
1512/1512 [==============================] - 16s 11ms/step - loss: 2.2294 - accuracy: 0.3869 - val_loss: 2.1643 - val_accuracy: 0.4180
Epoch 6/150
1512/1512 [==============================] - 16s 11ms/step - loss: 2.0781 - accuracy: 0.4451 - val_loss: 2.0418 - val_accuracy: 0.4788
Epoch 7/150
1512/1512 [==============================] - 16s 11ms/step 

In [0]:
loss, acc = model.evaluate(b_256, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

#Function for cutting the DCT output

In [0]:
def cut_dct(cut):
    a_cut = a_256[:,:cut,:cut,:]
    b_cut = b_256[:,:cut,:cut,:]
    model_big.fit(a_cut, y_train, batch_size=50, nb_epoch=100, verbose=1, validation_data=(b_cut, y_test))
    loss, acc = model_big.evaluate(b_cut, y_test, verbose=0)
    print('\n\n\n')
    print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [0]:
cut_dct(128)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1512 samples, validate on 378 samples
Epoch 1/100
1512/1512 [==============================] - 6s 4ms/step - loss: 3.2193 - accuracy: 0.0536 - val_loss: 3.1623 - val_accuracy: 0.1217
Epoch 2/100
1512/1512 [==============================] - 4s 3ms/step - loss: 2.9931 - accuracy: 0.1422 - val_loss: 2.7450 - val_accuracy: 0.2090
Epoch 3/100
1512/1512 [==============================] - 4s 3ms/step - loss: 2.6813 - accuracy: 0.2130 - val_loss: 2.4533 - val_accuracy: 0.2884
Epoch 4/100
1512/1512 [==============================] - 4s 3ms/step - loss: 2.4608 - accuracy: 0.2811 - val_loss: 2.2375 - val_accuracy: 0.3571
Epoch 5/100
1512/1512 [==============================] - 4s 3ms/step - loss: 2.2330 - accuracy: 0.3512 - val_loss: 2.0631 - val_accuracy: 0.4074
Epoch 6/100
1512/1512 [==============================] - 4s 3ms/step - loss: 2.0715 - accuracy: 0.4001 - val_loss: 2.0060 - val_accuracy: 0.4127
Epoch 7/100
1512/1512 [==============================] - 4s 3ms/step - loss: 1.9034

In [0]:
cut_dct(64)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1512 samples, validate on 378 samples
Epoch 1/100
1512/1512 [==============================] - 2s 2ms/step - loss: 3.1195 - accuracy: 0.0562 - val_loss: 3.0882 - val_accuracy: 0.0794
Epoch 2/100
1512/1512 [==============================] - 1s 861us/step - loss: 3.0173 - accuracy: 0.1091 - val_loss: 2.9611 - val_accuracy: 0.1587
Epoch 3/100
1512/1512 [==============================] - 1s 854us/step - loss: 2.8371 - accuracy: 0.1640 - val_loss: 2.6567 - val_accuracy: 0.2063
Epoch 4/100
1512/1512 [==============================] - 1s 864us/step - loss: 2.5735 - accuracy: 0.2235 - val_loss: 2.4079 - val_accuracy: 0.2434
Epoch 5/100
1512/1512 [==============================] - 1s 859us/step - loss: 2.4807 - accuracy: 0.2341 - val_loss: 2.4207 - val_accuracy: 0.2646
Epoch 6/100
1512/1512 [==============================] - 1s 866us/step - loss: 2.3987 - accuracy: 0.2533 - val_loss: 2.2363 - val_accuracy: 0.3228
Epoch 7/100
1512/1512 [==============================] - 1s 858us/step - 

In [0]:
cut_dct(32)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1512 samples, validate on 378 samples
Epoch 1/100
1512/1512 [==============================] - 1s 968us/step - loss: 3.0620 - accuracy: 0.0496 - val_loss: 3.0350 - val_accuracy: 0.0608
Epoch 2/100
1512/1512 [==============================] - 1s 374us/step - loss: 2.9991 - accuracy: 0.0999 - val_loss: 2.9858 - val_accuracy: 0.1138
Epoch 3/100
1512/1512 [==============================] - 1s 364us/step - loss: 2.9073 - accuracy: 0.1475 - val_loss: 2.8753 - val_accuracy: 0.1376
Epoch 4/100
1512/1512 [==============================] - 1s 366us/step - loss: 2.7163 - accuracy: 0.1627 - val_loss: 2.6490 - val_accuracy: 0.1376
Epoch 5/100
1512/1512 [==============================] - 1s 369us/step - loss: 2.5890 - accuracy: 0.1839 - val_loss: 2.5473 - val_accuracy: 0.1852
Epoch 6/100
1512/1512 [==============================] - 1s 367us/step - loss: 2.4736 - accuracy: 0.2348 - val_loss: 2.4485 - val_accuracy: 0.2354
Epoch 7/100
1512/1512 [==============================] - 1s 368us/step 

In [0]:
cut_dct(16)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1512 samples, validate on 378 samples
Epoch 1/100
1512/1512 [==============================] - 1s 340us/step - loss: 3.4398 - accuracy: 0.0575 - val_loss: 3.0056 - val_accuracy: 0.0582
Epoch 2/100
1512/1512 [==============================] - 0s 110us/step - loss: 2.9916 - accuracy: 0.0873 - val_loss: 2.9321 - val_accuracy: 0.1296
Epoch 3/100
1512/1512 [==============================] - 0s 110us/step - loss: 2.9290 - accuracy: 0.1085 - val_loss: 2.8552 - val_accuracy: 0.1587
Epoch 4/100
1512/1512 [==============================] - 0s 111us/step - loss: 2.8148 - accuracy: 0.1481 - val_loss: 2.7397 - val_accuracy: 0.2143
Epoch 5/100
1512/1512 [==============================] - 0s 102us/step - loss: 2.7228 - accuracy: 0.1872 - val_loss: 2.6378 - val_accuracy: 0.2328
Epoch 6/100
1512/1512 [==============================] - 0s 103us/step - loss: 2.6060 - accuracy: 0.2216 - val_loss: 2.5438 - val_accuracy: 0.2487
Epoch 7/100
1512/1512 [==============================] - 0s 105us/step 

#Augmentation

In [0]:
X_new = x_bck
Y_new = y_bck

In [0]:
X_new = np.array(X_new)
Y_new = np.array(Y_new)

In [0]:
X_new.shape

(1890, 3, 256, 256)

In [0]:
Y_new.shape

(1890, 1)

In [0]:
X_aug = np.zeros(X_new.shape)
Y_aug = np.zeros(Y_new.shape)

X_aug3 = np.zeros(X_new.shape)
X_aug4 = np.zeros(X_new.shape)

In [0]:
X_new[0].shape

(3, 256, 256)

In [0]:
from skimage.transform import rotate

In [0]:
X_aug1 = np.fliplr(X_new)
X_aug2 = np.flipud(X_new)
for i in range(len(X_new)):
    X_aug3[i] = rotate(X_new[i], angle = 45)
for i in range(len(X_new)):
    X_aug4[i] = rotate(X_new[i], angle = -45)

In [0]:
X_new = np.concatenate([X_new, X_aug1, X_aug2, X_aug3, X_aug4])
Y_new = np.concatenate([Y_new, Y_new, Y_new, Y_new, Y_new])

In [0]:
X_new.shape

(3780, 3, 256, 256)

#N-ary

In [18]:
a_256.shape

(1512, 256, 256, 3)

In [0]:
a_cut = a_256[:,:64,:64,:]
b_cut = b_256[:,:64,:64,:]

In [0]:
print(a_cut.shape)
print(b_cut.shape)
a_cut[0]

In [0]:
a_ary = a_cut.reshape((1512,4096,3))
b_ary = b_cut.reshape((378,4096,3))

In [82]:
print(a_ary.shape)
a_ary[0]

(1512, 4096, 3)


array([[ 1.29660828e+02,  1.29652069e+02,  1.29647507e+02],
       [-8.85041237e-01, -8.88628960e-01, -6.98852718e-01],
       [-4.85876016e-02,  1.18245995e-02,  1.33004924e-02],
       ...,
       [ 5.36340922e-02,  4.92457785e-02,  3.34869087e-01],
       [ 2.03092724e-01,  2.85183284e-02,  5.08773699e-02],
       [-1.30282655e-01,  1.45497113e-01,  9.04963687e-02]])

In [83]:
print(b_ary.shape)
b_ary[0]

(378, 4096, 3)


array([[ 1.23483917e+02,  1.23488960e+02,  1.23485229e+02],
       [-3.66153812e+00, -3.64887381e+00, -3.45911956e+00],
       [-1.94153726e-01,  2.10839007e-02,  3.79996859e-02],
       ...,
       [ 1.79408398e-02,  1.80211496e-02,  4.28981520e-02],
       [ 1.22897942e-02,  3.37779224e-02,  1.74096171e-02],
       [-1.58390030e-05, -1.36948545e-02,  4.36082184e-02]])

In [0]:
final_a_ary = np.zeros((a_ary.shape[0],4098,3))
final_b_ary = np.zeros((b_ary.shape[0],4098,3))

In [0]:
n = 2                                              #Change n for specific n-ary (n = anything in 2^n)
mul=int(a_ary.shape[1])/n
avgs = []
count=0

for j in range(a_ary.shape[0]):
    avgR = []
    avgG = []
    avgB = []
    for i in range(n):
        l = int(mul*i)
        lend = int(mul*(i+1) - 1)
        np.sort(a_ary[j,:,0])
        np.sort(a_ary[j,:,1])
        np.sort(a_ary[j,:,2])
        avgR.append(np.average(a_ary[j,l:lend,0]))
        avgG.append(np.average(a_ary[j,l:lend,1]))
        avgB.append(np.average(a_ary[j,l:lend,2]))
    avgR = np.array(avgR)
    avgG = np.array(avgG)
    avgB = np.array(avgB)
    final_a_ary[j,:,0] = np.concatenate((a_ary[j,:,0],avgR))
    final_a_ary[j,:,1] = np.concatenate((a_ary[j,:,1],avgG))
    final_a_ary[j,:,2] = np.concatenate((a_ary[j,:,2],avgB))

mul=int(b_ary.shape[1])/n

for j in range(b_ary.shape[0]):
    avgR = []
    avgG = []
    avgB = []
    for i in range(n):
        l = int(mul*i)
        lend = int(mul*(i+1) - 1)
        np.sort(b_ary[j,:,0])
        np.sort(b_ary[j,:,1])
        np.sort(b_ary[j,:,2])
        avgR.append(np.average(b_ary[j,l:lend,0]))
        avgG.append(np.average(b_ary[j,l:lend,1]))
        avgB.append(np.average(b_ary[j,l:lend,2]))
    avgR = np.array(avgR)
    avgG = np.array(avgG)
    avgB = np.array(avgB)
    final_b_ary[j,:,0] = np.concatenate((b_ary[j,:,0],avgR))
    final_b_ary[j,:,1] = np.concatenate((b_ary[j,:,1],avgG))
    final_b_ary[j,:,2] = np.concatenate((b_ary[j,:,2],avgB))

In [94]:
final_a_ary.shape

(1512, 4098, 3)

In [95]:
final_b_ary.shape

(378, 4098, 3)